In [1]:
import geocoder
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import re
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
APIKey= 'AhrrKF6BHmGIS3d71YtAb1AbGhXx5dBGhNgcWoS4vdd1apFoj1Spapvdq6r34UhC' 

In [3]:
g = geocoder.bing(None, addressline = '10011 Stonelake Boulevard, Austin, TX 78759', locality='Austin', adminDistrict='Texas', method='details', key=APIKey)
g.json
for result in  g:
    print(result.latlng)

[30.26497841, -97.74659729]


In [4]:
g = geocoder.bing([[30.476905, -97.774992], [47.845580, 3.321807]], method='batch_reverse', key=APIKey)

for result in g:
    print(result.address, result.city, result.postal, result.state, result.country)

13070 Ranch Road 620 N, Austin, TX 78717, United States Austin 78717 TX United States
D955, 89110 Chassy, France Chassy 89110 Bourgogne-Franche-Comté France


In [6]:
df = pd.read_excel('Addresses/addresses_25.10.2023.xlsx')

In [7]:
df.head(2)

,Unnamed: 0,name,address,amenities,street,city,state,zipcode,minprice,maxprice,minbed,maxbed,building_type,date
0,0,44 South,"4411 S Congress Ave, Austin, TX 78745","Dog & Cat Friendly, Fitness Center, Pool, In U...",4411 S Congress Ave,Austin,TX,78745,1399,1549,1,2,APT,2023-10-25 18:16:07
1,1,Brightleaf at Lakeline,"14115 N Highway 183, Austin, TX 78717","Dog & Cat Friendly, Fitness Center, Pool, Elev...",14115 N Highway 183,Austin,TX,78717,1460,1695,1,0,APT,2023-10-25 18:16:07


In [8]:
#dropping that first column with the index
df.drop(columns=['Unnamed: 0'], inplace=True)

In [9]:
#dropping all rows where the name is a null value
df = df.dropna(subset=['name'])

In [10]:
df['country']= 'United States'
df['lat'] = ' '
df['long'] = ' '

In [11]:
for i in df.index:
    try:
        g = geocoder.bing(
            None,
            addressline=df['street'][i],
            locality=df['city'][i],
            adminDistrict=df['state'][i],
            PostalCode=df['zipcode'][i],
            CountryRegion=df['country'][i],  # Correct field name: CountryRegion
            method='details',
            key='AqkhJ4a6kTQ6c4rm5vicHt7BsTav-KsWdaaKCnP794yFpzMDWPa1XCp14WUliC20'
        )

        # Access JSON data using 'g.json'
        if g.ok:
            df.loc[i, 'lat'] = g.latlng[0]
            df.loc[i, 'long'] = g.latlng[1]
        else:
            df.loc[i, 'lat'] = ''
            df.loc[i, 'long'] = ''
    except Exception as e:
        df.loc[i, 'lat'] = ''
        df.loc[i, 'long'] = ''
        print(f"Error geocoding address {i}: {e}")


In [12]:
df.head(5)

,name,address,amenities,street,city,state,zipcode,minprice,maxprice,minbed,maxbed,building_type,date,country,lat,long
0,44 South,"4411 S Congress Ave, Austin, TX 78745","Dog & Cat Friendly, Fitness Center, Pool, In U...",4411 S Congress Ave,Austin,TX,78745,1399,1549,1,2,APT,2023-10-25 18:16:07,United States,30.264978,-97.746597
1,Brightleaf at Lakeline,"14115 N Highway 183, Austin, TX 78717","Dog & Cat Friendly, Fitness Center, Pool, Elev...",14115 N Highway 183,Austin,TX,78717,1460,1695,1,0,APT,2023-10-25 18:16:07,United States,30.264978,-97.746597
2,The Upland,"12217 N Interstate 35, Austin, TX 78753","Dog & Cat Friendly, Fitness Center, Pool, In U...",12217 N Interstate 35,Austin,TX,78753,1401,1641,1,3,APT,2023-10-25 18:16:07,United States,30.264978,-97.746597
3,Water Oak,"12234 Heatherly Dr, Austin, TX 78747","Dog & Cat Friendly, Stainless Steel Appliances...",12234 Heatherly Dr,Austin,TX,78747,1305,1909,1,0,APT,2023-10-25 18:16:07,United States,30.264978,-97.746597
4,The Prescott,"8200 S Congress Ave, Austin, TX 78745","Dog & Cat Friendly, Fitness Center, Pool, In U...",8200 S Congress Ave,Austin,TX,78745,1524,2155,1,0,APT,2023-10-25 18:16:07,United States,30.264978,-97.746597


In [13]:
df2 = pd.read_excel('Addresses/stores.xlsx')

In [14]:
df2.head(3)

,type,name,address,city,state,zip,country
0,Grocery Store,Trader Joes,9722 Great Hills Trail,Austin,TX,78759,U.S
1,Grocery Store,Trader Joes,2805 Bee Cave Rd,Austin,TX,78746,U.S
2,Grocery Store,Trader Joes,"211 Walter Seaholm Dr, Ste 100",Austin,TX,78701,U.S


In [15]:
df2.rename(columns={'address': 'street'}, inplace=True)
df2['lat'] = ' '
df2['long'] = ' '

In [16]:
df2.head(5)

,type,name,street,city,state,zip,country,lat,long
0,Grocery Store,Trader Joes,9722 Great Hills Trail,Austin,TX,78759,U.S,,
1,Grocery Store,Trader Joes,2805 Bee Cave Rd,Austin,TX,78746,U.S,,
2,Grocery Store,Trader Joes,"211 Walter Seaholm Dr, Ste 100",Austin,TX,78701,U.S,,
3,Grocery Store,Sam's Club,4970 W. Highway 290,Austin,TX,78735,U.S,,
4,Grocery Store,Sam's Club,9900 S Interstate 35,Austin,TX,78748,U.S,,


In [17]:
for i in df2.index:
    try:
        g = geocoder.bing(
            None,
            addressline=df2['street'][i],
            locality=df2['city'][i],
            adminDistrict=df2['state'][i],
            PostalCode=df2['zip'][i],
            CountryRegion=df2['country '][i],  # Correct field name: CountryRegion
            method='details',
            key='AqkhJ4a6kTQ6c4rm5vicHt7BsTav-KsWdaaKCnP794yFpzMDWPa1XCp14WUliC20'
        )

        # Access JSON data using 'g.json'
        if g.ok:
            df2.loc[i, 'lat'] = g.latlng[0]
            df2.loc[i, 'long'] = g.latlng[1]
        else:
            df2.loc[i, 'lat'] = ''
            df2.loc[i, 'long'] = ''
    except Exception as e:
        df2.loc[i, 'lat'] = ''
        df2.loc[i, 'long'] = ''
        print(f"Error geocoding address {i}: {e}")


In [18]:
df2.head(5)

,type,name,street,city,state,zip,country,lat,long
0,Grocery Store,Trader Joes,9722 Great Hills Trail,Austin,TX,78759,U.S,30.264978,-97.746597
1,Grocery Store,Trader Joes,2805 Bee Cave Rd,Austin,TX,78746,U.S,30.264978,-97.746597
2,Grocery Store,Trader Joes,"211 Walter Seaholm Dr, Ste 100",Austin,TX,78701,U.S,30.264978,-97.746597
3,Grocery Store,Sam's Club,4970 W. Highway 290,Austin,TX,78735,U.S,30.264978,-97.746597
4,Grocery Store,Sam's Club,9900 S Interstate 35,Austin,TX,78748,U.S,30.264978,-97.746597
